## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [6]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [7]:
train, test = keras.datasets.cifar10.load_data()

In [8]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [9]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [10]:
from keras.layers import BatchNormalization

"""
建立神經網路，並加入 BN layer
"""
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [11]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [12]:
# 載入 Callbacks
"""Code Here
設定 callbacks: model checkpoint
"""
from keras.callbacks import ModelCheckpoint

model_ckpt = ModelCheckpoint(filepath="./saved_models/tmp.h5",
                            monitor="val_loss",save_best_only=True,
                            save_weights_only=True)

In [13]:
model = build_mlp(input_shape=x_train.shape[1:])
model.summary()
optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          shuffle=True,
          callbacks=[model_ckpt]
         )
model.save("./saved_models/final_model.h5")
model.save_weights("./saved_models/model_weights.h5")
# Collect results
train_loss = model.history.history["loss"]
valid_loss = model.history.history["val_loss"]
train_acc = model.history.history["accuracy"]
valid_acc = model.history.history["val_accuracy"]

pred_final = model.predict(x_test)
# Load back
model = keras.models.load_model("./saved_models/tmp.h5")
pred_loadback = model.predict(x_test)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
____________________________________________

Epoch 43/50
50000/50000 [==============================] - 3s 70us/step - loss: 0.5976 - accuracy: 0.8214 - val_loss: 1.5827 - val_accuracy: 0.4949
Epoch 44/50
50000/50000 [==============================] - 3s 63us/step - loss: 0.5804 - accuracy: 0.8273 - val_loss: 1.6136 - val_accuracy: 0.4920
Epoch 45/50
50000/50000 [==============================] - 3s 61us/step - loss: 0.5629 - accuracy: 0.8351 - val_loss: 1.6111 - val_accuracy: 0.4924
Epoch 46/50
50000/50000 [==============================] - 3s 62us/step - loss: 0.5453 - accuracy: 0.8435 - val_loss: 1.6461 - val_accuracy: 0.4905
Epoch 47/50
50000/50000 [==============================] - 3s 63us/step - loss: 0.5276 - accuracy: 0.8487 - val_loss: 1.6433 - val_accuracy: 0.4959
Epoch 48/50
50000/50000 [==============================] - 3s 70us/step - loss: 0.5117 - accuracy: 0.8550 - val_loss: 1.6756 - val_accuracy: 0.4926
Epoch 49/50
50000/50000 [==============================] - 3s 68us/step - loss: 0.4948 - accuracy: 0.8613 - val_

In [14]:
from sklearn.metrics import accuracy_score

final_model_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=pred_final.argmax(axis=-1))
loadback_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=pred_loadback.argmax(axis=-1))

print("Accuracy of final weights: %.3f" % final_model_acc)
print("Accuracy of best weights: %.3f" % loadback_acc)

Accuracy of final weights: 0.491
Accuracy of best weights: 0.491


In [19]:
new_model = build_mlp(input_shape=x_train.shape[1:])
new_model_pred = new_model.predict(x_test)
new_model_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=new_model_pred.argmax(axis=-1))
print("Accuracy of best weights: %.3f" % new_model_acc)

# new_model.load_weights("./saved_models/model_weights.h5")
# new_model_pred = new_model.predict(x_test)
# new_model_loadback_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=new_model_pred.argmax(axis=-1))
# print("Accuracy of model.save() loadback weights: %.3f" % new_model_loadback_acc)

new_model.load_weights("./saved_models/tmp.h5")
new_model_pred = new_model.predict(x_test)
new_model_loadback_acc = accuracy_score(y_true=y_test.argmax(axis=-1), y_pred=new_model_pred.argmax(axis=-1))
print("Accuracy of ModelCheckpoint(best only) loadback weights: %.3f" % new_model_loadback_acc)

Accuracy of best weights: 0.098
Accuracy of ModelCheckpoint(best only) loadback weights: 0.491


In [18]:
!jupyter nbconvert --to html Day_086_HW.ipynb Day_086_CB_ModelCheckPoint.ipynb

[NbConvertApp] Converting notebook Day_086_HW.ipynb to html
[NbConvertApp] Writing 378792 bytes to Day_086_HW.html
[NbConvertApp] Converting notebook Day_086_CB_ModelCheckPoint.ipynb to html
[NbConvertApp] Writing 413041 bytes to Day_086_CB_ModelCheckPoint.html
